Convert Categorical Data to Numerical


1.Load the Dataset





In [ ]:
import pandas as pd  


# Load dataset
df = pd.read_csv("C:/Users/Muqeem/OneDrive/Desktop/Final_Year_project/Data/MOCK_DATA_ORIGINAL.csv")

# Display first few rows
df.head()


,student_id,student_name,age,email,gpa,Difficulty_level,recommended_courses,College_Name,Course_Subject
0,1,Gillie Axtens,20,gaxtens0@slate.com,4.2,Intermediate,Front end Development,CIT,Data Science
1,2,Adrea Witherby,19,awitherby1@businessweek.com,7.7,Intermediate,Front end Development,Meenakshi,Data Science
2,3,Arleen Bellhanger,19,abellhanger2@accuweather.com,10.0,Intermediate,NaN,SRM,Cloud Computing
3,4,Annetta Stanyland,20,astanyland3@shop-pro.jp,8.5,Beginner,SQL,SIET,Data Science
4,5,Ferrel Atack,19,fatack4@newyorker.com,4.5,Beginner,Big Data Analytics,Crescent,Data Science


2.Handle Missing Values

In [1]:
# Check missing values
print(df.isnull().sum())

# Fill missing GPAs with the mean
df['gpa'].fillna(df['gpa'].mean(), inplace=True)

# Drop rows where essential data is missing
df.dropna(subset=['recommended_courses'], inplace=True)


NameError: name 'df' is not defined

3.Handle Duplicates

In [44]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)


 Convert Categorical Data to Numerical
📌 Goal: Convert non-numeric data for ML models.

Label Encoding for categorical values (if ordinal).
One-Hot Encoding for non-ordinal categorical values.

In [45]:
from sklearn.preprocessing import LabelEncoder

# Encode Difficulty Level (e.g., Easy → 0, Medium → 1, Hard → 2)
encoder = LabelEncoder()
df['Difficulty_level'] = encoder.fit_transform(df['Difficulty_level'])

# One-Hot Encoding for College Names
df = pd.get_dummies(df, columns=['College_Name'])


 Normalize Numerical Features
📌 Goal: Scale numerical values for better model performance.

Use MinMaxScaler (for values between 0-1) or StandardScaler (for normally distributed data).

In [46]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['gpa', 'age']] = scaler.fit_transform(df[['gpa', 'age']])


 Feature Engineering (Optional but Recommended)
📌 Goal: Create new meaningful features.

Engagement Score → Based on time spent on different topics.
Course Popularity Score → Count how often a course is recommended.

In [47]:
# Example: Add a Course Popularity Score
df['course_popularity'] = df.groupby('recommended_courses')['recommended_courses'].transform('count')


Save the Cleaned Dataset

In [48]:
df.to_csv("C:/Users/Muqeem/OneDrive/Desktop/Final_Year_project/Data/MOCK_DATA.csv", index=False)


 Identify Top-Tier Courses Based on Interests
Here’s how we can filter the best courses:

In [49]:
# def recommend_top_courses(student_id, df, top_n=5):
#     """ Recommend top-tier courses based on student interest. """
    
#     # Get the student's details
#     student = df[df['student_id'] == student_id]
    
#     if student.empty:
#         return "Student ID not found"
    
#     # Extract the student's Course_Subject interest & Difficulty Level
#     interest = student['Course_Subject'].values[0]
#     difficulty = student['Difficulty_level'].values[0]
    
#     # Filter top courses matching the same subject & difficulty
#     recommended_courses = df[
#         (df['Course_Subject'] == interest) & 
#         (df['Difficulty_level'] <= difficulty)  # Consider slightly easier courses too
#     ]
    
#     # Sort by highest GPA students taking the course (proxy for top-tier)
#     recommended_courses = recommended_courses.sort_values(by='gpa', ascending=False)
    
#     return recommended_courses[['recommended_courses', 'College_Name']].head(top_n)

# # Example Usage
# student_id = 102  # Change this to a real student ID
# top_courses = recommend_top_courses(student_id, df)
# print(top_courses)




# def recommend_top_courses(student_id, df, top_n=5):
#     """ Recommend top-tier courses based on student interest. """
    
#     # Get the student's details
#     student = df[df['student_id'] == student_id]
    
#     if student.empty:
#         return "Student ID not found"
    
#     # Extract the student's Course_Subject interest & Difficulty Level
#     interest = student['Course_Subject'].values[0]
#     difficulty = student['Difficulty_level'].values[0]
    
#     # Filter top courses matching the same subject & difficulty
#     recommended_courses = df[
#         (df['Course_Subject'] == interest) & 
#         (df['Difficulty_level'] <= difficulty)  
#     ]
    
#     # Check if results exist before proceeding
#     if recommended_courses.empty:
#         return "No matching courses found for this student."

#     # Sort by highest GPA students taking the course
#     recommended_courses = recommended_courses.sort_values(by='gpa', ascending=False)

#     # Ensure column exists before selection
#     available_columns = set(recommended_courses.columns)
#     required_columns = {'recommended_courses', 'College_Name'}
    
#     if not required_columns.issubset(available_columns):
#         return "Missing required columns in filtered data."

#     return recommended_courses[['recommended_courses', 'College_Name']].head(top_n)


In [54]:
def recommend_top_courses(student_id, df, top_n=5):
    """ Recommend top-tier courses based on student interest. """
    
    # Get the student's details
    student = df[df['student_id'] == student_id]
    
    if student.empty:
        return "Student ID not found"
    
    # Extract the student's Course_Subject interest & Difficulty Level
    interest = student['Course_Subject'].values[0]
    difficulty = student['Difficulty_level'].values[0]
    
    # Filter top courses matching the same subject & difficulty
    recommended_courses = df[
        (df['Course_Subject'] == interest) & 
        (df['Difficulty_level'] <= difficulty)  
    ]
    
    # Check if results exist before proceeding
    if recommended_courses.empty:
        return f"No matching courses found for student with ID {student_id}."

    # Sort by highest GPA students taking the course
    recommended_courses = recommended_courses.sort_values(by='gpa', ascending=False)
    
    # Ensure necessary columns exist before proceeding
    if 'recommended_courses' not in recommended_courses.columns or 'College_Name' not in recommended_courses.columns:
        return "Missing required columns in filtered data."

    # Return top N recommended courses
    return recommended_courses[['recommended_courses', 'College_Name']].head(top_n)

# Example Usage
student_id = 1  # Use a real student ID
top_courses = recommend_top_courses(student_id, df)
print(top_courses)


Missing required columns in filtered data.


In [55]:
import pandas as pd

# Assuming df is your DataFrame containing the data

# Count the number of occurrences of each course
course_counts = df['recommended_courses'].value_counts()

# Get the top 5 most popular courses
top_5_courses = course_counts.head(5)

# Display the top 5 popular courses and their count
print(top_5_courses)


recommended_courses
Big Data Analytics       138
AWS                      136
SQL                      130
Front end Development    126
Machine Learning         126
Name: count, dtype: int64
